# ML4SCI GSoC 2025 – ML4DQM Evaluation Test 1: ViT Classification Report

## Objective

Develop a machine learning model using a Vision Transformer (ViT) architecture to classify images (HCAL DigiOccupancy maps) based on their originating synthetic dataset (`Run357479` or `Run355456`).

---

## 1. Model Architecture & Hyperparameters

- **Model Type**: Vision Transformer (ViT) using **TensorFlow 2.17.1** + **Keras**
- **Input Shape**: `(64, 72, 1)` → `(ieta, iphi, channels)`
- **Patch Size**: `(8, 9)` → 64 non-overlapping patches
- **Projection Dimension**: `128` (per patch)
- **Transformer Layers**: `4`
- **Attention Heads**: `4`
- **MLP Dimension (per transformer block)**: `256`
- **Classifier Head**:
  - Global Average Pooling
  - Dropout (0.5)
  - Dense (64 units, ReLU)
  - Dropout (0.3)
  - Dense (1 unit, Sigmoid)

- **Total Parameters**: ~556,033 (all trainable)

---

##  2. Data Preprocessing

- **Datasets**:
  - `/kaggle/input/gsoc-test-data/Run357479_Dataset_iodic.npy`
  - `/kaggle/input/gsoc-test-data/Run355456_Dataset_jqkne.npy`
  - Shape: `(10000, 64, 72)` each

- **Steps**:
  - **Concatenation**: Combined to shape `(20000, 64, 72)`
  - **Labels**: `0` → Run357479, `1` → Run355456
  - **Reshape**: Add channel dim → `(20000, 64, 72, 1)`
  - **Split**: 
    - Train (60%): `12000` samples  
    - Validation (20%): `4000` samples  
    - Test (20%): `4000` samples  
    - **Stratified split**
  - **Normalization**:
    - Min-Max scaling to `[0, 1]` using only training set min (0.0) and max (~1564.9)
  - **Data Types**:
    - Images → `float32`
    - Labels → `int32`

---

##  3. Training

- **Framework**: TensorFlow 2.17.1 + Keras
- **Hardware**: 2× NVIDIA T4 GPUs via `tf.distribute.MirroredStrategy`

- **Data Pipeline**:
  - `tf.data.Dataset` with:
    - Caching
    - Shuffling (buffer size = 12000)
    - Batching (batch size = 64)
    - Prefetching

- **Optimizer**: AdamW (LR = `3e-4`, weight decay = `1e-5`)
- **Loss Function**: Binary Crossentropy
- **Metrics**:
  - Binary Accuracy
  - ROC AUC
  - PRC AUC

- **Callbacks**:
  - `ModelCheckpoint`: Save best model (`val_auc`)
  - `EarlyStopping`: Patience = 10, restore best weights

- **Training Duration**:
  - Stopped early at Epoch 13
  - Best model: Epoch 3 (`val_auc = 1.0`)
  - Total time: ~153 seconds

---

##  4. Evaluation Results

- **On Test Set**:
  - **Loss**: `0.0000`
  - **Accuracy**: `1.0000`
  - **ROC AUC**: `1.0000`
  - **PRC AUC**: `1.0000`

- **ROC Curve**:
  ![ROC Curve](roc_curve_test_set.png)

  > _Perfect ROC curve with AUC = 1.0_

---

##  5. Conclusion

The Vision Transformer achieved **perfect classification** performance:
- Accuracy = `1.0`
- ROC AUC = `1.0`

### Highlights:
- Synthetic datasets are clearly distinguishable via spatial occupancy patterns.
- ViT effectively captured those patterns.
- Training completed in a short time with high generalization performance.



By,
- Rhythm Suthar
- Email : rhythmsuthar123@gmail.com

---


In [ ]:
!pip install numpy tensorflow scikit-learn matplotlib wget requests # Added wget/requests for download
# Optional but recommended: A ViT implementation helper library
!pip install vit-keras # Example for Keras, or use Hugging Face Transformers

In [18]:
import numpy as np
import os

# Path for Run 357479 data
file_path_run357479 = '/kaggle/input/gsoc-test-data/Run357479_Dataset_iodic.npy'
# Path for Run 355456 data
file_path_run355456 = '/kaggle/input/gsoc-test-data/Run355456_Dataset_jqkne.npy'
# ---

print(f"Inspecting Run 357479 file: {file_path_run357479}")
data_run357479 = None # Initialize
try:
    # Check existence first
    if os.path.exists(file_path_run357479):
        data_run357479 = np.load(file_path_run357479)
        print(f"  - Successfully loaded.")
        print(f"  - Shape: {data_run357479.shape}")
        print(f"  - Data Type (dtype): {data_run357479.dtype}")
        
        # Calculate stats only if data is loaded and non-empty
        if data_run357479.size > 0:
            print(f"  - Minimum value: {np.min(data_run357479)}")
            print(f"  - Maximum value: {np.max(data_run357479)}")
            print(f"  - Mean value: {np.mean(data_run357479):.4f}")
            zero_percentage = (np.count_nonzero(data_run357479==0) / data_run357479.size) * 100
            print(f"  - Percentage of zero entries: {zero_percentage:.2f}%")
        else:
            print("  - Array is empty, skipping stats.")
            
        print("  - Sample slice [0, :3, :3]:")
        # Check dimensions and size before slicing
        if data_run357479.ndim >= 3 and data_run357479.shape[0] > 0 and data_run357479.shape[1] > 2 and data_run357479.shape[2] > 2:
             print(data_run357479[0, :3, :3])
        else:
            print("  - Not enough data/dimensions for sample slice.")
            
    else:
        print(f"  - Error: File not found at path: {file_path_run357479}")
        # Try listing parent dir for debugging
        parent_dir = os.path.dirname(file_path_run357479)
        print(f"  - Checking parent directory: {parent_dir}")
        try:
            if os.path.exists(parent_dir): print(f"    - Contents: {os.listdir(parent_dir)}")
            else: print(f"    - Parent directory does not exist.")
        except Exception as list_e: print(f"    - Error listing parent directory: {list_e}")
        
except Exception as e:
    print(f"  - Error loading or inspecting file: {e}")

print("-" * 30)

print(f"Inspecting Run 355456 file: {file_path_run355456}")
data_run355456 = None # Initialize
try:
    # Check existence first
    if os.path.exists(file_path_run355456):
        data_run355456 = np.load(file_path_run355456)
        print(f"  - Successfully loaded.")
        print(f"  - Shape: {data_run355456.shape}")
        print(f"  - Data Type (dtype): {data_run355456.dtype}")
        
        # Calculate stats only if data is loaded and non-empty
        if data_run355456.size > 0:
            print(f"  - Minimum value: {np.min(data_run355456)}")
            print(f"  - Maximum value: {np.max(data_run355456)}")
            print(f"  - Mean value: {np.mean(data_run355456):.4f}")
            zero_percentage = (np.count_nonzero(data_run355456==0) / data_run355456.size) * 100
            print(f"  - Percentage of zero entries: {zero_percentage:.2f}%")
        else:
            print("  - Array is empty, skipping stats.")
            
        print("  - Sample slice [0, :3, :3]:")
        # Check dimensions and size before slicing
        if data_run355456.ndim >= 3 and data_run355456.shape[0] > 0 and data_run355456.shape[1] > 2 and data_run355456.shape[2] > 2:
             print(data_run355456[0, :3, :3])
        else:
            print("  - Not enough data/dimensions for sample slice.")
            
    else:
        print(f"  - Error: File not found at path: {file_path_run355456}")
        # Try listing parent dir for debugging
        parent_dir = os.path.dirname(file_path_run355456)
        print(f"  - Checking parent directory: {parent_dir}")
        try:
            if os.path.exists(parent_dir): print(f"    - Contents: {os.listdir(parent_dir)}")
            else: print(f"    - Parent directory does not exist.")
        except Exception as list_e: print(f"    - Error listing parent directory: {list_e}")

except Exception as e:
    print(f"  - Error loading or inspecting file: {e}")

# Comparison
data1_loaded = data_run357479 is not None
data2_loaded = data_run355456 is not None

if data1_loaded and data2_loaded:
     print("-" * 30)
     print("Comparison:")
     # Check shapes only if both arrays actually have shapes
     if hasattr(data_run357479, 'shape') and hasattr(data_run355456, 'shape'):
         if data_run357479.shape == data_run355456.shape:
             print(f"  - Shapes match: {data_run357479.shape}")
         else:
             print(f"  - Shapes DIFFER: {data_run357479.shape} vs {data_run355456.shape}")
     # Check dtypes only if both arrays actually have dtypes
     if hasattr(data_run357479, 'dtype') and hasattr(data_run355456, 'dtype'):
         if data_run357479.dtype == data_run355456.dtype:
             print(f"  - Data types match: {data_run357479.dtype}")
         else:
             print(f"  - Data types DIFFER: {data_run357479.dtype} vs {data_run355456.dtype}")
elif data1_loaded or data2_loaded:
    print("-" * 30)
    print("Comparison skipped as only one file was loaded successfully.")
else:
    print("-" * 30)
    print("Comparison skipped as neither file loaded successfully.")

Inspecting Run 357479 file: /kaggle/input/gsoc-test-data/Run357479_Dataset_iodic.npy
  - Successfully loaded.
  - Shape: (10000, 64, 72)
  - Data Type (dtype): float64
  - Minimum value: 0.0
  - Maximum value: 1091.9733311864536
  - Mean value: 181.0826
  - Percentage of zero entries: 79.77%
  - Sample slice [0, :3, :3]:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
------------------------------
Inspecting Run 355456 file: /kaggle/input/gsoc-test-data/Run355456_Dataset_jqkne.npy
  - Successfully loaded.
  - Shape: (10000, 64, 72)
  - Data Type (dtype): float64
  - Minimum value: 0.0
  - Maximum value: 1564.944737802157
  - Mean value: 157.1423
  - Percentage of zero entries: 79.77%
  - Sample slice [0, :3, :3]:
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
------------------------------
Comparison:
  - Shapes match: (10000, 64, 72)
  - Data types match: float64


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import os


file_path_run357479 = '/kaggle/input/gsoc-test-data/Run357479_Dataset_iodic.npy'
file_path_run355456 = '/kaggle/input/gsoc-test-data/Run355456_Dataset_jqkne.npy'
# ---

print("Starting EDA Step 1: Value Distributions (Non-Zero)")

try:
    # Load data
    print("Loading data...")
    data_run357479 = np.load(file_path_run357479)
    data_run355456 = np.load(file_path_run355456)
    print("Data loaded.")

    # --- Analyze Run 357479 ---
    print("\nAnalyzing Run 357479...")
    # Flatten the array to get a 1D list of all values
    flat_data_357479 = data_run357479.flatten()
    # Select only non-zero values
    nonzero_data_357479 = flat_data_357479[flat_data_357479 > 0]
    
    if nonzero_data_357479.size > 0:
        print(f"  Found {nonzero_data_357479.size} non-zero values (out of {flat_data_357479.size}).")
        # Plot histogram
        plt.figure(figsize=(10, 6))
        plt.hist(nonzero_data_357479, bins=100, log=True) # Use 100 bins, log scale for y-axis
        plt.title('Distribution of Non-Zero DigiOccupancy Values (Run 357479)')
        plt.xlabel('DigiOccupancy Value')
        plt.ylabel('Frequency (Log Scale)')
        plt.grid(True, axis='y', linestyle='--')
        plt.savefig('histogram_run357479_nonzero.png')
        print("  Saved histogram to histogram_run357479_nonzero.png")
        plt.close() # Close the plot to prevent display issues if running many plots
    else:
        print("  No non-zero values found in Run 357479.")

    # --- Analyze Run 355456 ---
    print("\nAnalyzing Run 355456...")
    # Flatten the array
    flat_data_355456 = data_run355456.flatten()
    # Select only non-zero values
    nonzero_data_355456 = flat_data_355456[flat_data_355456 > 0]

    if nonzero_data_355456.size > 0:
        print(f"  Found {nonzero_data_355456.size} non-zero values (out of {flat_data_355456.size}).")
        # Plot histogram
        plt.figure(figsize=(10, 6))
        plt.hist(nonzero_data_355456, bins=100, log=True) # Use 100 bins, log scale for y-axis
        plt.title('Distribution of Non-Zero DigiOccupancy Values (Run 355456)')
        plt.xlabel('DigiOccupancy Value')
        plt.ylabel('Frequency (Log Scale)')
        plt.grid(True, axis='y', linestyle='--')
        plt.savefig('histogram_run355456_nonzero.png')
        print("  Saved histogram to histogram_run355456_nonzero.png")
        plt.close() # Close the plot
    else:
        print("  No non-zero values found in Run 355456.")

    print("\nEDA Step 1 finished.")

except FileNotFoundError:
    print("Error: One or both input files not found. Please ensure paths are correct.")
except Exception as e:
    print(f"An error occurred during EDA Step 1: {e}")

Starting EDA Step 1: Value Distributions (Non-Zero)
Loading data...
Data loaded.

Analyzing Run 357479...
  Found 9320000 non-zero values (out of 46080000).
  Saved histogram to histogram_run357479_nonzero.png

Analyzing Run 355456...
  Found 9320000 non-zero values (out of 46080000).
  Saved histogram to histogram_run355456_nonzero.png

EDA Step 1 finished.


In [19]:
import numpy as np
import matplotlib.pyplot as plt
import os

file_path_run357479 = '/kaggle/input/gsoc-test-data/Run357479_Dataset_iodic.npy'
file_path_run355456 = '/kaggle/input/gsoc-test-data/Run355456_Dataset_jqkne.npy'
# ---

print("Starting EDA Step 2: Average Spatial Patterns")

try:
    # Load data
    print("Loading data...")
    data_run357479 = np.load(file_path_run357479)
    data_run355456 = np.load(file_path_run355456)
    print("Data loaded.")

    # --- Calculate Average Maps ---
    print("Calculating average maps (mean across samples)...")
    # axis=0 averages over the 'LS' or 'sample' dimension
    mean_map_357479 = np.mean(data_run357479, axis=0)
    mean_map_355456 = np.mean(data_run355456, axis=0)
    print(f"  Shape of average maps: {mean_map_357479.shape}") # Should be (64, 72)

    # Determine a common color scale for comparison
    vmin = min(np.min(mean_map_357479), np.min(mean_map_355456))
    vmax = max(np.max(mean_map_357479), np.max(mean_map_355456))
    print(f"  Common color scale range: min={vmin:.2f}, max={vmax:.2f}")

    # --- Plot Average Map for Run 357479 ---
    print("Plotting average map for Run 357479...")
    plt.figure(figsize=(10, 8))
    im = plt.imshow(mean_map_357479, aspect='auto', cmap='viridis', vmin=vmin, vmax=vmax)
    plt.colorbar(im, label='Average DigiOccupancy')
    plt.title('Average DigiOccupancy Map (Run 357479)')
    plt.xlabel('iPhi Index')
    plt.ylabel('iEta Index')
    plt.savefig('average_map_run357479.png')
    print("  Saved average map to average_map_run357479.png")
    plt.close()

    # --- Plot Average Map for Run 355456 ---
    print("\nPlotting average map for Run 355456...")
    plt.figure(figsize=(10, 8))
    im = plt.imshow(mean_map_355456, aspect='auto', cmap='viridis', vmin=vmin, vmax=vmax) # Use same vmin/vmax
    plt.colorbar(im, label='Average DigiOccupancy')
    plt.title('Average DigiOccupancy Map (Run 355456)')
    plt.xlabel('iPhi Index')
    plt.ylabel('iEta Index')
    plt.savefig('average_map_run355456.png')
    print("  Saved average map to average_map_run355456.png")
    plt.close()

    print("\nEDA Step 2 finished.")

except FileNotFoundError:
    print("Error: One or both input files not found. Please ensure paths are correct.")
except Exception as e:
    print(f"An error occurred during EDA Step 2: {e}")

Starting EDA Step 2: Average Spatial Patterns
Loading data...
Data loaded.
Calculating average maps (mean across samples)...
  Shape of average maps: (64, 72)
  Common color scale range: min=0.00, max=1455.40
Plotting average map for Run 357479...
  Saved average map to average_map_run357479.png

Plotting average map for Run 355456...
  Saved average map to average_map_run355456.png

EDA Step 2 finished.


In [20]:
import numpy as np
import os


file_path_run357479 = '/kaggle/input/gsoc-test-data/Run357479_Dataset_iodic.npy'
file_path_run355456 = '/kaggle/input/gsoc-test-data/Run355456_Dataset_jqkne.npy'
# ---

print("Starting EDA Step 2b: Statistical Analysis of Average Spatial Patterns")

try:
    # Load data
    print("Loading data...")
    data_run357479 = np.load(file_path_run357479)
    data_run355456 = np.load(file_path_run355456)
    print("Data loaded.")

    # --- Calculate Average Maps ---
    print("\nCalculating average maps...")
    mean_map_357479 = np.mean(data_run357479, axis=0)
    mean_map_355456 = np.mean(data_run355456, axis=0)
    print("Average maps calculated.")

    # --- Analyze Average Map Stats ---
    print("\n--- Statistics for Average Map: Run 357479 ---")
    print(f"  Min: {np.min(mean_map_357479):.4f}")
    print(f"  Max: {np.max(mean_map_357479):.4f}")
    print(f"  Mean: {np.mean(mean_map_357479):.4f}")
    print(f"  Std Dev: {np.std(mean_map_357479):.4f}")
    print(f"  Median (50th %): {np.median(mean_map_357479):.4f}")
    print(f"  75th Percentile: {np.percentile(mean_map_357479, 75):.4f}")

    print("\n--- Statistics for Average Map: Run 355456 ---")
    print(f"  Min: {np.min(mean_map_355456):.4f}")
    print(f"  Max: {np.max(mean_map_355456):.4f}")
    print(f"  Mean: {np.mean(mean_map_355456):.4f}") # Should match overall mean from inspection
    print(f"  Std Dev: {np.std(mean_map_355456):.4f}")
    print(f"  Median (50th %): {np.median(mean_map_355456):.4f}")
    print(f"  75th Percentile: {np.percentile(mean_map_355456, 75):.4f}")

    # --- Find Top N Active Cells ---
    N_TOP = 5
    print(f"\n--- Top {N_TOP} Most Active Cells (Highest Average Occupancy) ---")
    
    # For Run 357479
    indices_flat_357479 = np.argsort(mean_map_357479.flatten())[-N_TOP:][::-1] # Get indices of top N, reversed for descending order
    coords_357479 = [np.unravel_index(i, mean_map_357479.shape) for i in indices_flat_357479]
    print("  Run 357479:")
    for i, coord in enumerate(coords_357479):
        print(f"    #{i+1}: iEta={coord[0]}, iPhi={coord[1]} -> Avg Value={mean_map_357479[coord]:.2f}")

    # For Run 355456
    indices_flat_355456 = np.argsort(mean_map_355456.flatten())[-N_TOP:][::-1]
    coords_355456 = [np.unravel_index(i, mean_map_355456.shape) for i in indices_flat_355456]
    print("  Run 355456:")
    for i, coord in enumerate(coords_355456):
        print(f"    #{i+1}: iEta={coord[0]}, iPhi={coord[1]} -> Avg Value={mean_map_355456[coord]:.2f}")


    # --- Analyze Difference Map ---
    print("\n--- Analysis of Difference Map (Run357479 - Run355456) ---")
    diff_map = mean_map_357479 - mean_map_355456
    
    print("  Difference Map Statistics:")
    print(f"    Min Diff: {np.min(diff_map):.4f}")
    print(f"    Max Diff: {np.max(diff_map):.4f}")
    print(f"    Mean Diff: {np.mean(diff_map):.4f}") # Run357479_mean - Run355456_mean
    print(f"    Std Dev Diff: {np.std(diff_map):.4f}")
    
    # Overall difference magnitude
    mean_abs_diff = np.mean(np.abs(diff_map))
    print(f"    Mean Absolute Difference: {mean_abs_diff:.4f}")

    # Find Top N Differences
    print(f"\n  Top {N_TOP} Largest Positive Differences (Run357479 > Run355456):")
    indices_flat_diff_pos = np.argsort(diff_map.flatten())[-N_TOP:][::-1]
    coords_diff_pos = [np.unravel_index(i, diff_map.shape) for i in indices_flat_diff_pos]
    for i, coord in enumerate(coords_diff_pos):
        print(f"    #{i+1}: iEta={coord[0]}, iPhi={coord[1]} -> Diff={diff_map[coord]:.2f} (Vals: {mean_map_357479[coord]:.2f} vs {mean_map_355456[coord]:.2f})")

    print(f"\n  Top {N_TOP} Largest Negative Differences (Run357479 < Run355456):")
    indices_flat_diff_neg = np.argsort(diff_map.flatten())[:N_TOP] # Get indices of smallest N
    coords_diff_neg = [np.unravel_index(i, diff_map.shape) for i in indices_flat_diff_neg]
    for i, coord in enumerate(coords_diff_neg):
        print(f"    #{i+1}: iEta={coord[0]}, iPhi={coord[1]} -> Diff={diff_map[coord]:.2f} (Vals: {mean_map_357479[coord]:.2f} vs {mean_map_355456[coord]:.2f})")


    print("\nEDA Step 2b finished.")

except FileNotFoundError:
    print("Error: One or both input files not found. Please ensure paths are correct.")
except Exception as e:
    print(f"An error occurred during EDA Step 2b: {e}")

Starting EDA Step 2b: Statistical Analysis of Average Spatial Patterns
Loading data...
Data loaded.

Calculating average maps...
Average maps calculated.

--- Statistics for Average Map: Run 357479 ---
  Min: 0.0000
  Max: 948.9780
  Mean: 181.0826
  Std Dev: 360.7005
  Median (50th %): 0.0000
  75th Percentile: 0.0000

--- Statistics for Average Map: Run 355456 ---
  Min: 0.0000
  Max: 1455.4012
  Mean: 157.1423
  Std Dev: 363.9954
  Median (50th %): 0.0000
  75th Percentile: 0.0000

--- Top 5 Most Active Cells (Highest Average Occupancy) ---
  Run 357479:
    #1: iEta=6, iPhi=30 -> Avg Value=948.98
    #2: iEta=6, iPhi=22 -> Avg Value=948.96
    #3: iEta=57, iPhi=20 -> Avg Value=948.87
    #4: iEta=6, iPhi=70 -> Avg Value=948.84
    #5: iEta=57, iPhi=12 -> Avg Value=948.77
  Run 355456:
    #1: iEta=59, iPhi=44 -> Avg Value=1455.40
    #2: iEta=59, iPhi=4 -> Avg Value=1439.71
    #3: iEta=59, iPhi=26 -> Avg Value=1436.12
    #4: iEta=59, iPhi=8 -> Avg Value=1432.68
    #5: iEta=59, i

In [21]:
import numpy as np
import os

file_path_run357479 = '/kaggle/input/gsoc-test-data/Run357479_Dataset_iodic.npy'
file_path_run355456 = '/kaggle/input/gsoc-test-data/Run355456_Dataset_jqkne.npy'
# ---

# --- Configuration ---
N_SAMPLES_TO_COMPARE = 3
# ---

print(f"Starting EDA Step 3: Comparing {N_SAMPLES_TO_COMPARE} Individual Slice Examples Statistically")

try:
    # Load data
    print("Loading data...")
    # Ensure we load the full arrays, not just memory-mapped objects if that was tried before
    data_run357479 = np.load(file_path_run357479)
    data_run355456 = np.load(file_path_run355456)
    print("Data loaded.")

    # Get number of samples and ensure it's at least N_SAMPLES_TO_COMPARE
    num_samples = data_run357479.shape[0]
    if num_samples < N_SAMPLES_TO_COMPARE:
        print(f"Warning: Only {num_samples} samples available, reducing comparison count.")
        N_SAMPLES_TO_COMPARE = num_samples

    # Generate random indices
    np.random.seed(42) # for reproducibility
    random_indices = np.random.choice(num_samples, N_SAMPLES_TO_COMPARE, replace=False)
    print(f"Comparing slices at indices: {random_indices}")

    # Loop through random indices
    for i, sample_idx in enumerate(random_indices):
        print(f"\n--- Comparing Slice at Index {sample_idx} ---")
        
        # Extract slices
        slice_357479 = data_run357479[sample_idx] # Shape (64, 72)
        slice_355456 = data_run355456[sample_idx] # Shape (64, 72)

        # --- Stats for Slice from Run 357479 ---
        print("  Run 357479 Slice Stats:")
        if slice_357479.size > 0:
            print(f"    Min: {np.min(slice_357479):.2f}")
            print(f"    Max: {np.max(slice_357479):.2f}")
            print(f"    Mean: {np.mean(slice_357479):.2f}")
            zeros_perc = (np.count_nonzero(slice_357479==0) / slice_357479.size) * 100
            print(f"    Zeros: {zeros_perc:.2f}%")
        else: print("    Slice empty.")

        # --- Stats for Slice from Run 355456 ---
        print("  Run 355456 Slice Stats:")
        if slice_355456.size > 0:
            print(f"    Min: {np.min(slice_355456):.2f}")
            print(f"    Max: {np.max(slice_355456):.2f}")
            print(f"    Mean: {np.mean(slice_355456):.2f}")
            zeros_perc = (np.count_nonzero(slice_355456==0) / slice_355456.size) * 100
            print(f"    Zeros: {zeros_perc:.2f}%")
        else: print("    Slice empty.")

        # --- Stats for Difference (Slice 357479 - Slice 355456) ---
        if slice_357479.size > 0 and slice_355456.size > 0:
            print("  Difference (Slice 357479 - Slice 355456) Stats:")
            diff_slice = slice_357479 - slice_355456
            print(f"    Min Diff: {np.min(diff_slice):.2f}")
            print(f"    Max Diff: {np.max(diff_slice):.2f}")
            print(f"    Mean Diff: {np.mean(diff_slice):.2f}")
            mean_abs_diff = np.mean(np.abs(diff_slice))
            print(f"    Mean Absolute Diff: {mean_abs_diff:.2f}")
        else: print("  Difference calculation skipped.")

    print("\nEDA Step 3 finished.")

except FileNotFoundError:
    print("Error: One or both input files not found. Please ensure paths are correct.")
except Exception as e:
    print(f"An error occurred during EDA Step 3: {e}")

Starting EDA Step 3: Comparing 3 Individual Slice Examples Statistically
Loading data...
Data loaded.
Comparing slices at indices: [6252 4684 1731]

--- Comparing Slice at Index 6252 ---
  Run 357479 Slice Stats:
    Min: 0.00
    Max: 940.44
    Mean: 178.80
    Zeros: 79.77%
  Run 355456 Slice Stats:
    Min: 0.00
    Max: 1456.67
    Mean: 155.47
    Zeros: 79.77%
  Difference (Slice 357479 - Slice 355456) Stats:
    Min Diff: -518.84
    Max Diff: 565.15
    Mean Diff: 23.33
    Mean Absolute Diff: 70.17

--- Comparing Slice at Index 4684 ---
  Run 357479 Slice Stats:
    Min: 0.00
    Max: 950.15
    Mean: 182.54
    Zeros: 79.77%
  Run 355456 Slice Stats:
    Min: 0.00
    Max: 1447.28
    Mean: 156.57
    Zeros: 79.77%
  Difference (Slice 357479 - Slice 355456) Stats:
    Min Diff: -501.70
    Max Diff: 588.11
    Mean Diff: 25.97
    Mean Absolute Diff: 71.80

--- Comparing Slice at Index 1731 ---
  Run 357479 Slice Stats:
    Min: 0.00
    Max: 957.50
    Mean: 184.00
    Zero

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
import os
import gc

file_path_run357479 = '/kaggle/input/gsoc-test-data/Run357479_Dataset_iodic.npy'
file_path_run355456 = '/kaggle/input/gsoc-test-data/Run355456_Dataset_jqkne.npy'
# ---

try:
    print("Loading data for preprocessing...")
    # Label 0: Run 357479
    # Label 1: Run 355456
    data_label0 = np.load(file_path_run357479)
    data_label1 = np.load(file_path_run355456)
    print("Data loaded successfully.")

    # --- 1. Combine Data ---
    X = np.concatenate((data_label0, data_label1), axis=0)
    print(f"\n1. Combined data shape (X): {X.shape}")

    # --- 2. Create Labels ---
    num_samples_label0 = data_label0.shape[0]
    num_samples_label1 = data_label1.shape[0]
    labels0 = np.zeros(num_samples_label0, dtype=np.int32) # Use int32
    labels1 = np.ones(num_samples_label1, dtype=np.int32)  # Use int32
    y = np.concatenate((labels0, labels1), axis=0)
    print(f"2. Combined labels shape (y): {y.shape}")
    print(f"   Label examples: {y[::5000]}")

    # --- 3. Reshape for ViT ---
    X = np.expand_dims(X, axis=-1)
    print(f"3. Reshaped data shape (added channel): {X.shape}")

    # Clean up original large arrays now before splitting
    del data_label0, data_label1
    gc.collect()

    # --- 4. Train/Validation/Test Split (BEFORE Normalization) ---
    print("\n4. Splitting data into Train, Validation, Test sets...")
    # Split ratio: 60% train, 20% validation, 20% test
    X_train_val, X_test, y_train_val, y_test = train_test_split(
        X, y,
        test_size=0.20,       # 20% for testing
        random_state=42,     # For reproducibility
        stratify=y           # Keep label proportions
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_val, y_train_val,
        test_size=0.25,       # 25% of 80% = 20% of total for validation
        random_state=42,     # For reproducibility
        stratify=y_train_val # Stratify again
    )
    # Clean up intermediate split
    del X, y, X_train_val, y_train_val
    gc.collect()

    print(f"   Raw Split Shapes:")
    print(f"     Train set:      X={X_train.shape}, y={y_train.shape}")
    print(f"     Validation set: X={X_val.shape}, y={y_val.shape}")
    print(f"     Test set:       X={X_test.shape}, y={y_test.shape}")

    # --- 5. Normalization (Fit on Train, Apply to All) ---
    print("\n5. Normalizing data (Min-Max scaling, fitted on Train set)...")
    
    # Calculate min and max *only* on the training data
    min_val = np.min(X_train)
    max_val = np.max(X_train)
    print(f"   Train set Min value: {min_val}")
    print(f"   Train set Max value: {max_val}")

    # Apply the scaling to all sets using train min/max
    if max_val > min_val:
        X_train = (X_train - min_val) / (max_val - min_val)
        X_val = (X_val - min_val) / (max_val - min_val)
        X_test = (X_test - min_val) / (max_val - min_val)
        
        # Clip values outside [0, 1] just in case (can happen with test/val data)
        X_val = np.clip(X_val, 0.0, 1.0)
        X_test = np.clip(X_test, 0.0, 1.0)
        
        print(f"   Train data normalized. New min={np.min(X_train):.1f}, max={np.max(X_train):.1f}")
        print(f"   Val data normalized.   New min={np.min(X_val):.1f}, max={np.max(X_val):.1f}")
        print(f"   Test data normalized.  New min={np.min(X_test):.1f}, max={np.max(X_test):.1f}")
    else:
        print("   Skipping normalization as training data range is zero.")

    # --- 6. Data Type Conversion ---
    print("\n6. Converting data arrays to float32...")
    X_train = X_train.astype(np.float32)
    X_val = X_val.astype(np.float32)
    X_test = X_test.astype(np.float32)
    # Labels should already be int32 from step 2

    print(f"   Final dtypes: X_train={X_train.dtype}, y_train={y_train.dtype}")
    print(f"   Final shapes:")
    print(f"     Train set:      X={X_train.shape}, y={y_train.shape}")
    print(f"     Validation set: X={X_val.shape}, y={y_val.shape}")
    print(f"     Test set:       X={X_test.shape}, y={y_test.shape}")

    print("\nPreprocessing finished. Data splits (X_train, y_train, etc.) are ready.")
    

except FileNotFoundError:
    print("Error: One or both input files not found. Please ensure paths are correct.")
except Exception as e:
    print(f"An error occurred during preprocessing: {e}")

Loading data for preprocessing...
Data loaded successfully.

1. Combined data shape (X): (20000, 64, 72)
2. Combined labels shape (y): (20000,)
   Label examples: [0 0 1 1]
3. Reshaped data shape (added channel): (20000, 64, 72, 1)

4. Splitting data into Train, Validation, Test sets...
   Raw Split Shapes:
     Train set:      X=(12000, 64, 72, 1), y=(12000,)
     Validation set: X=(4000, 64, 72, 1), y=(4000,)
     Test set:       X=(4000, 64, 72, 1), y=(4000,)

5. Normalizing data (Min-Max scaling, fitted on Train set)...
   Train set Min value: 0.0
   Train set Max value: 1564.944737802157
   Train data normalized. New min=0.0, max=1.0
   Val data normalized.   New min=0.0, max=1.0
   Test data normalized.  New min=0.0, max=1.0

6. Converting data arrays to float32...
   Final dtypes: X_train=float32, y_train=int32
   Final shapes:
     Train set:      X=(12000, 64, 72, 1), y=(12000,)
     Validation set: X=(4000, 64, 72, 1), y=(4000,)
     Test set:       X=(4000, 64, 72, 1), y=(40

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np # Keep numpy imported

# --- Define Hyperparameters ---
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 72
NUM_CHANNELS = 1
IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CHANNELS)

PATCH_SIZE = (8, 9) # Results in 8x8=64 patches
NUM_PATCHES = (IMAGE_HEIGHT // PATCH_SIZE[0]) * (IMAGE_WIDTH // PATCH_SIZE[1])
PROJECTION_DIM = 128 # Embedding dimension
NUM_HEADS = 4 # Number of attention heads
TRANSFORMER_LAYERS = 4 # Number of transformer blocks
MLP_HEAD_UNITS = [64] # Classification head dense layers
NUM_CLASSES = 1 # Binary classification output

print("--- Hyperparameters ---")
print(f"Input Image Shape: {IMAGE_SHAPE}")
print(f"Patch Size: {PATCH_SIZE}")
print(f"Number of Patches: {NUM_PATCHES}")
print(f"Projection Dim (Embedding Size): {PROJECTION_DIM}")
print(f"Number of Attention Heads: {NUM_HEADS}")
print(f"Number of Transformer Layers: {TRANSFORMER_LAYERS}")
print(f"MLP Head Units: {MLP_HEAD_UNITS}")
print(f"Number of Output Classes: {NUM_CLASSES}")
print("-----------------------")

# --- Layer Definitions ---

# Patch Creation Layer
class Patches(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size[0], self.patch_size[1], 1],
            strides=[1, self.patch_size[0], self.patch_size[1], 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches
        
    # Need get_config for saving/loading models with custom layers
    def get_config(self):
        config = super().get_config()
        config.update({"patch_size": self.patch_size})
        return config

# Patch Encoding Layer (Learnable projection + Position embedding)
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded
        
    # Need get_config for saving/loading models with custom layers
    def get_config(self):
        config = super().get_config()
        config.update({"num_patches": self.num_patches, "projection_dim": self.projection.units})
        return config

# --- ViT Builder Function ---
def build_vit_classifier(input_shape, patch_size, num_patches, projection_dim,
                         num_heads, transformer_layers, mlp_head_units, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        # Ensure key_dim is compatible with projection_dim and num_heads
        if projection_dim % num_heads != 0:
             raise ValueError(f"projection_dim ({projection_dim}) must be divisible by num_heads ({num_heads})")
        key_dim = projection_dim // num_heads
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=key_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP inside transformer block
        # Standard practice: expand to 2x or 4x projection_dim then contract back
        mlp_intermediate_dim = projection_dim * 2 
        x3 = layers.Dense(units=mlp_intermediate_dim, activation=tf.nn.gelu)(x3)
        x3 = layers.Dropout(0.1)(x3)
        x3 = layers.Dense(units=projection_dim)(x3) # No activation on final MLP layer in block? Check common practice. Usually yes. Let's add GELU.
        x3 = layers.Activation(tf.nn.gelu)(x3)
        x3 = layers.Dropout(0.1)(x3)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Final part: Classification Head
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    # Global average pooling across patches
    representation = layers.GlobalAveragePooling1D()(representation)
    representation = layers.Dropout(0.5)(representation) # Increased dropout before final head

    # MLP Head layers
    features = representation
    for units in mlp_head_units:
        features = layers.Dense(units=units, activation=tf.nn.relu)(features) # Using ReLU here
        features = layers.Dropout(0.3)(features) # Dropout within head

    # Final classification layer
    logits = layers.Dense(num_classes, activation="sigmoid")(features) # Sigmoid for binary output

    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

# --- Build the Model ---
print("\nBuilding the ViT model...")
# Ensure variables defined in the preprocessing step are available if needed for input_shape
# Or redefine input_shape directly as we know it
input_shape_for_model = IMAGE_SHAPE

# Handle potential errors during build
try:
    vit_classifier = build_vit_classifier(
        input_shape=input_shape_for_model,
        patch_size=PATCH_SIZE,
        num_patches=NUM_PATCHES,
        projection_dim=PROJECTION_DIM,
        num_heads=NUM_HEADS,
        transformer_layers=TRANSFORMER_LAYERS,
        mlp_head_units=MLP_HEAD_UNITS,
        num_classes=NUM_CLASSES
    )
    
    # Print the model summary
    print("\nModel Summary:")
    vit_classifier.summary()
    print("\nViT model built successfully.")

except ValueError as ve:
    print(f"\nError building model: {ve}")
    print("Please check hyperparameters (e.g., projection_dim divisibility by num_heads).")
except Exception as e:
    print(f"\nAn unexpected error occurred during model building: {e}")

--- Hyperparameters ---
Input Image Shape: (64, 72, 1)
Patch Size: (8, 9)
Number of Patches: 64
Projection Dim (Embedding Size): 128
Number of Attention Heads: 4
Number of Transformer Layers: 4
MLP Head Units: [64]
Number of Output Classes: 1
-----------------------

Building the ViT model...

Model Summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 64, 72, 1)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ patches (Patches)         │ (None, None, 72)       │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ patch_encoder             │ (None, 64, 128)        │         17,536 │ patches[0][0]          │
│ (PatchEncoder)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 64, 128)        │            256 │ patch_encoder[0][0]    │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 64, 128)        │         66,048 │ layer_normalization[0… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 64, 128)        │              0 │ multi_head_attention[… │
│                           │                        │                │ patch_encoder[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 64, 128)        │            256 │ add[0][0]              │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64, 256)        │         33,024 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 64, 256)        │              0 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 64, 128)        │         32,896 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 64, 128)        │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 64, 128)        │              0 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 64, 128)        │              0 │ dropout_2[0][0],       │
│                           │                        │                │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 64, 128)        │            256 │ add_1[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 64, 128)        │         66,048 │ layer_normalization_2… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)          

 Total params: 556,033 (2.12 MB)

 Trainable params: 556,033 (2.12 MB)

 Non-trainable params: 0 (0.00 B)


ViT model built successfully.


In [13]:
import tensorflow as tf
from tensorflow import keras

import time # To time training
import gc

print(tf.__version__) # Check TensorFlow version

# --- Configuration ---
EPOCHS = 50 
BATCH_SIZE_PER_REPLICA = 32 # Adjust based on GPU memory if needed
LEARNING_RATE = 3e-4 # Starting learning rate for AdamW

# --- 1. Distribution Strategy ---
try:
    strategy = tf.distribute.MirroredStrategy()
    NUM_GPUS = strategy.num_replicas_in_sync
    print(f'Number of devices: {NUM_GPUS}')
    if NUM_GPUS < 2:
        print("Warning: MirroredStrategy found fewer than 2 devices. Check GPU detection.")
except Exception as e:
    print("Error setting up MirroredStrategy, falling back to default strategy.")
    print(f"Error: {e}")
    strategy = tf.distribute.get_strategy() # Default strategy
    NUM_GPUS = 1

GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_GPUS
print(f'Global Batch Size: {GLOBAL_BATCH_SIZE}')

# --- 2. Data Pipeline ---
# Buffer size for shuffling (should be larger than batch size, ideally size of dataset for perfect shuffle)
BUFFER_SIZE = len(X_train) 

print("\nCreating tf.data Datasets...")
# Training Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.cache() # Cache data in memory after first epoch
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(GLOBAL_BATCH_SIZE)
train_dataset = train_dataset.prefetch(tf.data.AUTOTUNE) # Prefetch batches

# Validation Dataset
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_dataset = val_dataset.cache() # Cache validation data
val_dataset = val_dataset.batch(GLOBAL_BATCH_SIZE) # Use global batch size for validation too
val_dataset = val_dataset.prefetch(tf.data.AUTOTUNE)

print("Training and Validation Datasets created.")
print(f"  Training dataset element spec: {train_dataset.element_spec}")


# --- 3. Build & Compile Model within Strategy Scope ---
print("\nBuilding and Compiling Model under Strategy Scope...")
with strategy.scope():
    
    try:
         # Use previously defined hyperparameters
         vit_classifier_dist = build_vit_classifier(
             input_shape=IMAGE_SHAPE, patch_size=PATCH_SIZE, num_patches=NUM_PATCHES,
             projection_dim=PROJECTION_DIM, num_heads=NUM_HEADS, transformer_layers=TRANSFORMER_LAYERS,
             mlp_head_units=MLP_HEAD_UNITS, num_classes=NUM_CLASSES
         )
         
         # Compile the model
         optimizer = tf.keras.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=1e-5)
         vit_classifier_dist.compile(
             optimizer=optimizer,
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=[
                 tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                 tf.keras.metrics.AUC(name='auc'),
                 tf.keras.metrics.AUC(name='prc', curve='PR') # Area under Precision-Recall curve
             ]
         )
         print("Model built and compiled successfully.")
         vit_classifier_dist.summary() # Optionally print summary again
         
    except NameError as ne:
         print(f"Error: Ensure 'build_vit_classifier' function and layers are defined before this block. {ne}")
         # Handle error gracefully, maybe exit or raise
         raise ne
    except Exception as e:
         print(f"An unexpected error occurred during model build/compile: {e}")
         raise e


# --- 4. Callbacks ---
print("\nSetting up Callbacks...")
# Path where checkpoint will be saved
checkpoint_filepath = 'vit_model_checkpoint.keras'

# Save the best model based on validation AUC
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False, # Save the whole model
    monitor='val_auc',    # Monitor validation AUC
    mode='max',           # Maximize AUC
    save_best_only=True)  # Only save if improved

# Stop training early if validation AUC doesn't improve for 'patience' epochs
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc',
    patience=10,          # Number of epochs with no improvement after which training will be stopped.
    mode='max',
    restore_best_weights=True, # Restore model weights from the epoch with the best value of the monitored quantity.
    verbose=1
)


callbacks_list = [
    model_checkpoint_callback,
    early_stopping_callback,
]
print("Callbacks defined: ModelCheckpoint (on val_auc), EarlyStopping (on val_auc)")

# Clean up potentially large variables before training
# del X_train, y_train, X_val, y_val # Keep if needed later, but tf.data should have them
gc.collect()

# --- 5. Training ---
print("\n--- Starting Model Training ---")
start_time = time.time()

history = vit_classifier_dist.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset,
    callbacks=callbacks_list
)

end_time = time.time()
print(f"--- Training Finished ---")
print(f"Total Training Time: {(end_time - start_time):.2f} seconds")


# Find the epoch with the best validation AUC
best_epoch = np.argmax(history.history['val_auc'])
best_val_auc = np.max(history.history['val_auc'])
print(f"\nBest Validation AUC: {best_val_auc:.4f} at Epoch: {best_epoch + 1}")

2.17.1
Number of devices: 2
Global Batch Size: 64

Creating tf.data Datasets...
Training and Validation Datasets created.
  Training dataset element spec: (TensorSpec(shape=(None, 64, 72, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))

Building and Compiling Model under Strategy Scope...
Model built and compiled successfully.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 64, 72, 1)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ patches_1 (Patches)       │ (None, None, 72)       │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ patch_encoder_1           │ (None, 64, 128)        │         17,536 │ patches_1[0][0]        │
│ (PatchEncoder)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_9     │ (None, 64, 128)        │            256 │ patch_encoder_1[0][0]  │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_4    │ (None, 64, 128)        │         66,048 │ layer_normalization_9… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_9… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_8 (Add)               │ (None, 64, 128)        │              0 │ multi_head_attention_… │
│                           │                        │                │ patch_encoder_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_10    │ (None, 64, 128)        │            256 │ add_8[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 64, 256)        │         33,024 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_15 (Dropout)      │ (None, 64, 256)        │              0 │ dense_12[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 64, 128)        │         32,896 │ dropout_15[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_4 (Activation) │ (None, 64, 128)        │              0 │ dense_13[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_16 (Dropout)      │ (None, 64, 128)        │              0 │ activation_4[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_9 (Add)               │ (None, 64, 128)        │              0 │ dropout_16[0][0],      │
│                           │                        │                │ add_8[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_11    │ (None, 64, 128)        │            256 │ add_9[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_5    │ (None, 64, 128)        │         66,048 │ layer_normalization_1… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_1… │
├──────────────────────

 Total params: 556,033 (2.12 MB)

 Trainable params: 556,033 (2.12 MB)

 Non-trainable params: 0 (0.00 B)


Setting up Callbacks...
Callbacks defined: ModelCheckpoint (on val_auc), EarlyStopping (on val_auc)

--- Starting Model Training ---
Epoch 1/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 37s 62ms/step - accuracy: 0.8755 - auc: 0.9352 - loss: 0.2425 - prc: 0.9350 - val_accuracy: 0.9990 - val_auc: 0.9993 - val_loss: 0.0086 - val_prc: 0.9987
Epoch 2/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - accuracy: 0.9991 - auc: 0.9993 - loss: 0.0089 - prc: 0.9986 - val_accuracy: 0.9980 - val_auc: 0.9980 - val_loss: 0.0144 - val_prc: 0.9960
Epoch 3/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - accuracy: 0.9994 - auc: 0.9994 - loss: 0.0057 - prc: 0.9988 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 4.1727e-05 - val_prc: 1.0000
Epoch 4/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 48ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 5.5825e-04 - prc: 1.0000 - val_accuracy: 1.0000 - val_auc: 1.0000 - val_loss: 2.0585e-05 - val_prc: 1.0000
Epoch 5/50
188/188 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - accuracy: 1.0000 - auc:

In [17]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import gc
from sklearn.metrics import roc_curve, auc # For ROC curve later
import matplotlib.pyplot as plt # For ROC curve later


class Patches(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = tuple(patch_size)

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size[0], self.patch_size[1], 1],
            strides=[1, self.patch_size[0], self.patch_size[1], 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        num_patches_h = tf.shape(images)[1] // self.patch_size[0]
        num_patches_w = tf.shape(images)[2] // self.patch_size[1]
        num_patches_total = num_patches_h * num_patches_w
        patches = tf.reshape(patches, [batch_size, num_patches_total, patch_dims])
        return patches

    def get_config(self):
        config = super().get_config()
        config.update({"patch_size": self.patch_size})
        return config

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        # Define layers within __init__ for proper tracking
        # Use names that likely match how they were saved
        self.projection = layers.Dense(units=projection_dim, name='projection')
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim, name='position_embedding' # Check name consistency
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

    def get_config(self):
        config = super().get_config()
        config.update({"num_patches": self.num_patches, "projection_dim": self.projection_dim})
        return config


# --- Evaluation on Test Set ---
print("\n--- Evaluating Best Model on Test Set ---")


try:
    if 'X_test' not in locals() or 'y_test' not in locals():
         saved_splits_path = 'processed_splits.npz'
         if os.path.exists(saved_splits_path):
             print(f"Test data (X_test, y_test) not found in memory. Loading from {saved_splits_path}...")
             with np.load(saved_splits_path) as data:
                 X_test = data['X_test']
                 y_test = data['y_test']
             print("Test data loaded.")
         else:
             raise NameError("Test data (X_test, y_test) not found. Please ensure preprocessing output is loaded or saved file exists.")
    else:
        print("Using existing X_test, y_test from memory.")

    # Define path to the saved model checkpoint
    checkpoint_filepath = 'vit_model_checkpoint.keras'

    if os.path.exists(checkpoint_filepath):
        print(f"Loading best model from: {checkpoint_filepath}")

       
        custom_objects = {"Patches": Patches, "PatchEncoder": PatchEncoder}
        loaded_model = tf.keras.models.load_model(
            checkpoint_filepath,
            custom_objects=custom_objects
        )

        print("Model loaded successfully.")

        # Re-compile the loaded model
        loaded_model.compile(
             loss=tf.keras.losses.BinaryCrossentropy(),
             metrics=[
                 tf.keras.metrics.BinaryAccuracy(name='accuracy'),
                 tf.keras.metrics.AUC(name='auc'),
                 tf.keras.metrics.AUC(name='prc', curve='PR')
             ]
        )
        print("Model re-compiled for evaluation.")

        # Create tf.data.Dataset for test set
        try:
            if 'GLOBAL_BATCH_SIZE' not in locals(): GLOBAL_BATCH_SIZE = 64
        except NameError: GLOBAL_BATCH_SIZE = 64

        test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
        test_dataset = test_dataset.batch(GLOBAL_BATCH_SIZE)
        test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

        print(f"\nEvaluating on Test Set (using Batch Size: {GLOBAL_BATCH_SIZE})...")
        results = loaded_model.evaluate(test_dataset, verbose=1)

        print("\n--- Test Set Results ---")
        if results and len(results) > 3:
            test_loss = results[0]
            test_accuracy = results[1]
            test_auc = results[2]
            test_prc = results[3]
            print(f"Test Loss:     {test_loss:.4f}")
            print(f"Test Accuracy: {test_accuracy:.4f}")
            print(f"Test AUC:      {test_auc:.4f}")
            print(f"Test PRC (AUC):{test_prc:.4f}")
        else:
            print(f"Evaluation returned unexpected result format: {results}")

    else:
        print(f"Error: Model checkpoint file not found at {checkpoint_filepath}")
        print("Please ensure training ran and saved the checkpoint.")

except NameError as ne:
     print(f"Error accessing data or variables: {ne}")
except FileNotFoundError as fnfe:
     print(f"Error: A required file was not found: {fnfe}")
except Exception as e:
    # Print detailed traceback for unexpected errors
    import traceback
    print(f"An error occurred during test set evaluation:")
    traceback.print_exc()


try:
    if 'loaded_model' in locals() and 'X_test' in locals() and 'y_test' in locals():
        print("\nGenerating ROC curve for the test set...")
        y_pred_proba = loaded_model.predict(X_test)

        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
        roc_auc = auc(fpr, tpr) # Calculate AUC from fpr, tpr

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ViT ROC curve (AUC = {roc_auc:.4f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Chance (AUC = 0.5)')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate (FPR)')
        plt.ylabel('True Positive Rate (TPR)')
        plt.title('Receiver Operating Characteristic (ROC) Curve - Test Set')
        plt.legend(loc="lower right")
        plt.grid(True)
        plt.savefig('roc_curve_test_set.png')
        print("ROC curve saved to roc_curve_test_set.png")
        plt.close()
    else:
        print("\nSkipping ROC curve generation as model or test data is not available.")
except Exception as e:
    print(f"\nAn error occurred during ROC curve generation: {e}")


--- Evaluating Best Model on Test Set ---
Using existing X_test, y_test from memory.
Loading best model from: vit_model_checkpoint.keras


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:391: UserWarning: `build()` was called on layer 'patch_encoder_1', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Model loaded successfully.
Model re-compiled for evaluation.

Evaluating on Test Set (using Batch Size: 64)...
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - accuracy: 1.0000 - auc: 1.0000 - loss: 3.8766e-05 - prc: 1.0000

--- Test Set Results ---
Test Loss:     0.0000
Test Accuracy: 1.0000
Test AUC:      1.0000
Test PRC (AUC):1.0000

Generating ROC curve for the test set...
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
ROC curve saved to roc_curve_test_set.png


In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import gc


class Patches(layers.Layer):
    def __init__(self, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.patch_size = tuple(patch_size)

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size[0], self.patch_size[1], 1],
            strides=[1, self.patch_size[0], self.patch_size[1], 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        num_patches_h = tf.shape(images)[1] // self.patch_size[0]
        num_patches_w = tf.shape(images)[2] // self.patch_size[1]
        num_patches_total = num_patches_h * num_patches_w
        patches = tf.reshape(patches, [batch_size, num_patches_total, patch_dims])
        return patches

    def get_config(self):
        config = super().get_config()
        config.update({"patch_size": self.patch_size})
        return config

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patches = num_patches
        self.projection_dim = projection_dim
        # Define layers within __init__ for proper tracking
        self.projection = layers.Dense(units=projection_dim, name='projection')
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim, name='position_embedding' # Check name consistency
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

    def get_config(self):
        config = super().get_config()
        config.update({"num_patches": self.num_patches, "projection_dim": self.projection_dim})
        return config


# --- Show Predictions vs Actuals ---
print("\n--- Showing Predictions vs Actuals for Test Set Samples ---")

N_SAMPLES_TO_SHOW = 15 # Number of samples to display

try:
    # Check if necessary variables exist, reload if needed
    if 'loaded_model' not in locals():
        print("Model 'loaded_model' not found. Attempting to load...")
        checkpoint_filepath = 'vit_model_checkpoint.keras'
        if os.path.exists(checkpoint_filepath):
             # *** Using custom_objects dictionary for loading - NO DECORATORS NEEDED ***
             custom_objects_dict = {"Patches": Patches, "PatchEncoder": PatchEncoder}
             loaded_model = tf.keras.models.load_model(
                 checkpoint_filepath,
                 custom_objects=custom_objects_dict # Pass the dictionary here
             )
             print("Model loaded successfully using custom_objects.")
        else:
             raise NameError("Model checkpoint not found and 'loaded_model' not in memory.")

    if 'X_test' not in locals() or 'y_test' not in locals():
        print("Test data (X_test, y_test) not found. Attempting to load...")
        saved_splits_path = 'processed_splits.npz' # Adjust if needed
        if os.path.exists(saved_splits_path):
             with np.load(saved_splits_path) as data:
                 X_test = data['X_test']
                 y_test = data['y_test']
             print("Test data loaded.")
        else:
             raise NameError("Test data not available.")

    actual_samples_in_test = len(X_test)
    if actual_samples_in_test < N_SAMPLES_TO_SHOW:
        print(f"Warning: Only {actual_samples_in_test} samples in test set, showing all.")
        N_SAMPLES_TO_SHOW = actual_samples_in_test

    if N_SAMPLES_TO_SHOW == 0:
         print("No samples available in the test set to show.")
    else:
        X_test_subset = X_test[:N_SAMPLES_TO_SHOW]
        y_test_subset = y_test[:N_SAMPLES_TO_SHOW]

        print(f"\nGetting predictions for the first {N_SAMPLES_TO_SHOW} test samples...")
        try:
             if 'GLOBAL_BATCH_SIZE' not in locals(): GLOBAL_BATCH_SIZE = 64
        except NameError: GLOBAL_BATCH_SIZE = 64

        y_pred_proba_subset = loaded_model.predict(X_test_subset, batch_size=GLOBAL_BATCH_SIZE)
        y_pred_labels_subset = (y_pred_proba_subset > 0.5).astype(int).flatten()

        # Print results
        print("\n-----------------------------------------------------")
        print("| Idx | True Label | Predicted Probability | Pred Label |")
        print("-----------------------------------------------------")
        for i in range(N_SAMPLES_TO_SHOW):
            true_label = y_test_subset[i]
            pred_proba = y_pred_proba_subset[i][0] if y_pred_proba_subset.ndim > 1 else y_pred_proba_subset[i]
            pred_label = y_pred_labels_subset[i]
            # Format probability using scientific notation for very small/large values
            print(f"| {i:<3} | {true_label:<10} | {pred_proba:<21.5e} | {pred_label:<10} |")
        print("-----------------------------------------------------")

except NameError as ne:
     print(f"Error: Necessary variable not found. {ne}")
except FileNotFoundError as fnfe:
     print(f"Error: A required file was not found. {fnfe}")
except Exception as e:
    import traceback
    print(f"An error occurred:")
    traceback.print_exc()


--- Showing Predictions vs Actuals for Test Set Samples ---

Getting predictions for the first 15 test samples...
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step

-----------------------------------------------------
| Idx | True Label | Predicted Probability | Pred Label |
-----------------------------------------------------
| 0   | 1          | 9.99972e-01           | 1          |
| 1   | 0          | 4.82919e-05           | 0          |
| 2   | 0          | 4.78464e-05           | 0          |
| 3   | 1          | 9.99972e-01           | 1          |
| 4   | 0          | 4.78514e-05           | 0          |
| 5   | 0          | 4.77857e-05           | 0          |
| 6   | 1          | 9.99972e-01           | 1          |
| 7   | 0          | 4.83697e-05           | 0          |
| 8   | 1          | 9.99972e-01           | 1          |
| 9   | 1          | 9.99972e-01           | 1          |
| 10  | 1          | 9.99972e-01           | 1          |
| 11  | 1          | 9.99972e-01           |

#  Reflections and Potential Extensions on Test 1 Results

I’ve successfully completed the primary objective for **Test 1**, developing a **Vision Transformer (ViT)** that achieved **perfect classification** (Accuracy = 1.0, AUC = 1.0) on the test set using the provided **synthetic HCAL DigiOccupancy data**. The model was trained efficiently using a distributed strategy across multiple GPUs.

---

##  Understanding the Perfect Score

While achieving a perfect score is certainly encouraging, it's important to interpret it in context:

- The **synthetic nature** of the data might make the classes **perfectly separable**.
- The model achieved perfect validation metrics **within just 3 epochs**, indicating strong signal separation.
- In a real-world scenario, such results might warrant concerns around **data leakage** or overly simple patterns, but here it likely reflects the characteristics of the generated datasets.

---

##  Potential Next Steps to Stand Out

To go beyond the core objectives and demonstrate a **deeper understanding and initiative**, I could pursue the following extensions:

---

### 1.  Model Interpretability (Attention Maps)

- Visualize **attention maps** from the ViT to understand which regions (iEta/iPhi) contribute most to classification.
- This would involve:
  - Extracting **attention weights** from the transformer blocks.
  - Overlaying attention maps on the input images or visualizing them separately per layer/head.
- This step could **highlight spatial features** that the model focuses on, linking them to observed differences (e.g., hotspot locations in EDA).

---

### 2.  Baseline Model Comparison

To understand the **added value of ViT**, implement and evaluate **simpler models** as baselines:

- **Basic CNN** with a few convolutional layers.
- **Logistic Regression** on:
  - Flattened images, or  
  - Aggregated statistics (e.g., max/mean occupancy per slice).

> This would help determine if the **transformer’s complexity** was necessary or if simpler models could achieve similar performance.

---

### 3.  Enhanced Report Discussion

Include a dedicated section in the final report:

- Discuss the **implications of a perfect score**.
- Acknowledge that performance on **synthetic data** might not generalize to real-world detector data.
- Emphasize the **limitations** and **generalization challenges** expected with real data.

---

### 4.  Further Exploration (Optional)

#### a. Mixture-of-Experts (MoE) ViT

- Explore the **MoE-ViT** architecture mentioned in the prompt.
- Compare:
  - **Parameter efficiency**
  - **Training speed**
  - **Generalization capability** vs. standard ViT

#### b. Ablation / Sensitivity Analysis

- Train a **smaller ViT**:
  - Fewer layers
  - Fewer heads
- Assess how much capacity is **really needed** to solve this classification task.

---

## Summary

By integrating steps like:

- **Attention map visualization**
- **Baseline model benchmarking**
- **Critical analysis of synthetic data limitations**

I can deliver a **well-rounded** and **insightful** submission that goes beyond just meeting the minimum criteria and reflects a deeper engagement with the problem.

---
